In [7]:
!pip3 install tweepy

In [8]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import seaborn as sns
import tweepy
import requests
import time
import re
import datetime

from datetime import date
from datetime import datetime
from functools import reduce

%matplotlib inline

# Gathering

In [9]:
#load 'twitter-archice-enchanced.csv'
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
twitter_archive.sample(15)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1498,692417313023332352,NaN,NaN,2016-01-27 18:42:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Herschel. He's slightly bigger than ur av...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/692417313...,7,10,Herschel,None,None,pupper,None
951,751950017322246144,NaN,NaN,2016-07-10 01:23:49 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Lola. She's a surfing pupper. 13/10 ma...,NaN,NaN,NaN,https://vine.co/v/5WrjaYAMvMO,13,10,Lola,None,None,pupper,None
1861,675483430902214656,NaN,NaN,2015-12-12 01:12:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Rare shielded battle dog here. Very happy abou...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675483430...,5,10,None,None,None,None,None
1662,682962037429899265,NaN,NaN,2016-01-01 16:30:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darrel. He just robbed a 7/11 and is i...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/682962037...,7,11,Darrel,None,None,None,None
1903,674638615994089473,NaN,NaN,2015-12-09 17:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This pupper is fed up with being tickled. 12/1...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674638615...,12,10,None,None,None,pupper,None
598,798686750113755136,NaN,NaN,2016-11-16 00:38:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: Say hello to Jack (pronounced ""...",6.833919e+17,4.196984e+09,2016-01-02 20:58:09 +0000,https://twitter.com/dog_rates/status/683391852...,11,10,Jack,None,None,None,None
1909,674422304705744896,NaN,NaN,2015-12-09 02:56:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ava. She doesn't understand flowers. 1...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674422304...,12,10,Ava,None,None,None,None
1876,675111688094527488,NaN,NaN,2015-12-11 00:35:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Oliver. He thought what was insid...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675111688...,8,10,Oliver,None,None,None,None
1091,737445876994609152,NaN,NaN,2016-05-31 00:49:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Just wanted to share this super rare Rainbow F...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/737445876...,13,10,None,None,floofer,None,None
113,870726314365509632,8.707262e+17,16487760.0,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None


In [10]:
#Download image-predictions
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url) #image-requests
with open(os.path.join('image-predictions.tsv'), mode = 'wb') as file:
    file.write(response.content)

In [11]:
#load image-predictions file
images = pd.read_csv('image-predictions.tsv', sep = '\t')
images.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [12]:
#setting Twitter API
#consumer_key = '...'
#consumer_secret = '...'
#access_token = '...'
#access_secret = '...'

#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)

#api = tweepy.API(auth, parser = tweepy.parsers.JSONParser())

In [13]:
# Store data to tweet_json.txt
#with open('tweet_json.txt', mode = 'w') as file:
    #json.dump(tweets_data, file)

In [14]:
# Loading the Twitter API data
#twitter_counts_df = pd.read_json('tweet_json.txt')
#twitter_counts_df['tweet_id'] = tweet_success
#twitter_counts_df = twitter_counts_df[['tweet_id', 'favorite_count', 'retweet_count']]

In [15]:
# Read the saved tweet_json.txt file into a dataframe
x = pd.read_json('tweet-json.txt', lines=True)
x

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
0,2017-08-01 16:23:56+00:00,892420643555336193,892420643555336192,This is Phineas. He's a mystical boy. Only eve...,False,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,39467,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
1,2017-08-01 00:17:27+00:00,892177421306343426,892177421306343424,This is Tilly. She's just checking pup on you....,False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,33819,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
2,2017-07-31 00:18:03+00:00,891815181378084864,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,False,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,25461,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
3,2017-07-30 15:58:51+00:00,891689557279858688,891689557279858688,This is Darla. She commenced a snooze mid meal...,False,"[0, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891689552724799489, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,42908,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
4,2017-07-29 16:00:24+00:00,891327558926688256,891327558926688256,This is Franklin. He would like you to stop ca...,False,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891327551943041024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,41048,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2349,2015-11-16 00:24:50+00:00,666049248165822465,666049248165822464,Here we have a 1949 1st generation vulpix. Enj...,False,"[0, 120]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666049244999131136, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,111,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
2350,2015-11-16 00:04:52+00:00,666044226329800704,666044226329800704,This is a purebred Piers Morgan. Loves to Netf...,False,"[0, 137]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666044217047650304, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,311,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
2351,2015-11-15 23:21:54+00:00,666033412701032449,666033412701032448,Here is a very happy pup. Big fan of well-main...,False,"[0, 130]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666033409081393153, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,128,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
2352,2015-11-15 23:05:30+00:00,666029285002620928,666029285002620928,This is a western brown Mitsubishi terrier. Up...,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 666029276303482880, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,132,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN


In [16]:
# Loading the Twitter API data
twitter_counts_df = pd.read_json('tweet-json.txt', lines=True)
#favcount,retcount,twid=[],[],[]
#for res in twitter_counts_df['extended_entities']:
 #   if(type(res) != float and len(res['media']) > 0):
  #      twid.append(res['media'][0]['id'])
   # else:
    #    twid.append("NaN")

#favcount=twitter_counts_df['favorite_count'].values.tolist()
#retcount=twitter_counts_df['retweet_count'].values.tolist()

#twitter_counts_df=pd.DataFrame({'tweet_id':twid,'Favorite_Count':favcount,'Retweet_Count':retcount})
#twitter_counts_df
twitter_counts_df = twitter_counts_df[['id','favorite_count', 'retweet_count']]
twitter_counts_df=pd.DataFrame(twitter_counts_df, columns = ['id', 'favorite_count','retweet_count'])
twitter_counts_df=twitter_counts_df.rename(columns={'id': 'tweet_id'})
twitter_counts_df


,tweet_id,favorite_count,retweet_count
0,892420643555336193,39467,8853
1,892177421306343426,33819,6514
2,891815181378084864,25461,4328
3,891689557279858688,42908,8964
4,891327558926688256,41048,9774
...,...,...,...
2349,666049248165822465,111,41
2350,666044226329800704,311,147
2351,666033412701032449,128,47
2352,666029285002620928,132,48


# Access
Assessing data programmaticaly


### twitter_archive Analysis:

In [17]:
twitter_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
17,888804989199671297,NaN,NaN,2017-07-22 16:56:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zeke. He has a new stick. Very proud o...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/888804989...,13,10,Zeke,None,None,None,None
597,798694562394996736,NaN,NaN,2016-11-16 01:09:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: I'm not sure what's happening h...,7.012147e+17,4.196984e+09,2016-02-21 01:19:47 +0000,https://twitter.com/dog_rates/status/701214700...,12,10,None,None,None,None,None
1225,714141408463036416,NaN,NaN,2016-03-27 17:25:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I know we only rate dogs, but since it's Easte...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/714141408...,10,10,None,None,None,None,None
1516,690959652130045952,NaN,NaN,2016-01-23 18:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This golden is happy to refute the soft mouth ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/690959652...,11,10,None,None,None,None,None
262,842765311967449089,NaN,NaN,2017-03-17 15:51:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Indie. She's not a fan of baths but she's...,NaN,NaN,NaN,"https://www.gofundme.com/get-indie-home/,https...",12,10,Indie,None,None,None,None


In [101]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [102]:
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [103]:
twitter_archive[twitter_archive.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [104]:
twitter_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [105]:
twitter_archive.isnull().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [106]:
twitter_archive['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [107]:
twitter_archive['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [108]:
twitter_archive['name'].value_counts()

None        745
a            55
Charlie      12
Oliver       11
Cooper       11
           ... 
Florence      1
Stormy        1
Arlo          1
Ambrose       1
Nico          1
Name: name, Length: 957, dtype: int64

In [109]:
# count the entries for each source
twitter_archive.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [110]:
twitter_archive['text'].value_counts()

This is Charlie. He's a West Side Niddlewog. Mucho fluffy. 12/10 would pet so damn well https://t.co/B9dOrmnPVt                                 1
This is Trigger. He was minding his own business on stair when he overheard someone say they don't like bacon. 11/10 https://t.co/yqohZK4CL0    1
Today, 10/10, should be National Dog Rates Day                                                                                                  1
Meet Bailey. She plays with her food. Very childish. Doesn't even need a battle helmet smh. Still cute though. 9/10 https://t.co/CLEOjxhTEx     1
This is Stella. She's happier than I will ever be. 10/10 would trade lives with https://t.co/JSs2bfDtTS                                         1
                                                                                                                                               ..
This is Gilbert. He's being chased by a battalion of miniature floof cows. 10/10 we all believe in you Gilbert https://t.co/

In [111]:
#Unrecogized names
twitter_archive[twitter_archive['name'].apply(len) < 3]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
56,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a pupper approaching maximum borkdrive...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004...,14,10,a,None,None,pupper,None
393,825876512159186944,NaN,NaN,2017-01-30 01:21:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Mo. No one will push him around in the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/825876512...,11,10,Mo,None,None,None,None
446,819015337530290176,NaN,NaN,2017-01-11 02:57:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Bo. He was a very good ...,8.190048e+17,4.196984e+09,2017-01-11 02:15:36 +0000,https://twitter.com/dog_rates/status/819004803...,14,10,Bo,doggo,None,None,None
449,819004803107983360,NaN,NaN,2017-01-11 02:15:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo. He was a very good First Doggo. 14...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/819004803...,14,10,Bo,doggo,None,None,None
553,804026241225523202,NaN,NaN,2016-11-30 18:16:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo. He's going to make me cry. 13/10 p...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/804026241...,13,10,Bo,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2349,666051853826850816,NaN,NaN,2015-11-16 00:35:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an odd dog. Hard on the outside but lo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666051853...,2,10,an,None,None,None,None
2350,666050758794694657,NaN,NaN,2015-11-16 00:30:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a truly beautiful English Wilson Staff...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666050758...,10,10,a,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None


### Images Analysis:

In [112]:
images.sample(15)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1183,738537504001953792,https://pbs.twimg.com/media/Cj_P7rSUgAAYQbz.jpg,1,chow,0.808737,True,gibbon,0.028942,False,Pembroke,2.649790e-02,True
413,673956914389192708,https://pbs.twimg.com/media/CVpgPGwWoAEV7gG.jpg,1,pug,0.586161,True,Brabancon_griffon,0.082744,True,Chihuahua,4.587770e-02,True
333,672160042234327040,https://pbs.twimg.com/media/CVP9_beUEAAwURR.jpg,1,pug,0.561027,True,French_bulldog,0.222114,True,Labrador_retriever,6.545560e-02,True
1022,710272297844797440,https://pbs.twimg.com/media/Cdtk414WoAIUG0v.jpg,1,Old_English_sheepdog,0.586307,True,wire-haired_fox_terrier,0.118622,True,Lakeland_terrier,1.068060e-01,True
1128,727685679342333952,https://pbs.twimg.com/media/ChlCQg-VIAQ_8g4.jpg,1,Border_collie,0.462408,True,collie,0.214556,True,Eskimo_dog,3.560360e-02,True
128,668274247790391296,https://pbs.twimg.com/media/CUYv4d2WUAAziXs.jpg,1,soft-coated_wheaten_terrier,0.406374,True,Lakeland_terrier,0.263854,True,toy_poodle,1.508440e-01,True
1510,786036967502913536,https://pbs.twimg.com/media/CtKHLuCWYAA2TTs.jpg,1,golden_retriever,0.993830,True,cocker_spaniel,0.003143,True,Great_Pyrenees,9.174140e-04,True
161,668932921458302977,https://pbs.twimg.com/media/CUiG6_ZXAAAPaw_.jpg,1,standard_poodle,0.237638,True,Old_English_sheepdog,0.195573,True,toy_poodle,1.446580e-01,True
1272,750011400160841729,https://pbs.twimg.com/media/CmfmvGUWgAAuVKD.jpg,1,muzzle,0.237620,False,Boston_bull,0.087150,True,sombrero,6.850990e-02,False
1523,788150585577050112,https://pbs.twimg.com/media/CvASw6dWcAQmo3X.jpg,3,chow,0.814145,True,Pomeranian,0.112704,True,Chihuahua,1.588320e-02,True


In [113]:
#Just figuring out how to get Jupyter Notebook to show images

from IPython.display import display, Image
Image(url = 'https://pbs.twimg.com/media/DFWra-3VYAA2piG.jpg')
Image(url = 'https://pbs.twimg.com/media/DE8yicJW0AAAvBJ.jpg')
# This is an image for tweet_id 745057283344719872

##### Little cute Dog :D 

In [114]:
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [115]:
images.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [116]:
images['p1'].value_counts()

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
                     ... 
pencil_box              1
cheetah                 1
clog                    1
sandbar                 1
boathouse               1
Name: p1, Length: 378, dtype: int64

In [34]:
images.isnull().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

### Twitter Counts Analysis:

In [35]:
twitter_counts_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2354 non-null   int64
 1   favorite_count  2354 non-null   int64
 2   retweet_count   2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


In [36]:
twitter_counts_df.isnull().sum()

tweet_id          0
favorite_count    0
retweet_count     0
dtype: int64

In [37]:
twitter_counts_df['tweet_id'].value_counts()

749075273010798592    1
741099773336379392    1
798644042770751489    1
825120256414846976    1
769212283578875904    1
                     ..
794926597468000259    1
673705679337693185    1
673342308415348736    1
754482103782404096    1
700151421916807169    1
Name: tweet_id, Length: 2354, dtype: int64

In [38]:
twitter_counts_df.describe()

,tweet_id,favorite_count,retweet_count
count,2.354000e+03,2354.000000,2354.000000
mean,7.426978e+17,8080.968564,3164.797366
std,6.852812e+16,11814.771334,5284.770364
min,6.660209e+17,0.000000,0.000000
25%,6.783975e+17,1415.000000,624.500000
50%,7.194596e+17,3603.500000,1473.500000
75%,7.993058e+17,10122.250000,3652.000000
max,8.924206e+17,132810.000000,79515.000000


In [39]:
twitter_counts_df.duplicated().sum()

0

## Assessing Summary

### Twitter Archive/

*Completeness:*
- missing data  within those columns: 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls'
- tweet_id is an integer  (applies in all tables)

*Validity:*
- some dogs have names like 'None', 'a', or 'an.' and Found an instance of a name being "O" instead of "O'Malley"
- some entries are retweets
- Sources difficult to read.

*Accuracy:*
- timestamp is an object
- wrong data types ("in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id", "timestamp" and "retweeted_status_timestamp") should be changing from float & objects to integers and time
- rating_numerator goes up to 1776

*Consistency:*
- rating_denominator should be a standard 10, but there are a multitude of other values
- the source column still has the HTML tags

### Images

*Validity:*
- p1, p2 and p3 columns have invalid data... the algorithm labeled a dog photo as a seat belt, or mailbox amd others?

*Consistency:*
- p1, p2, p3 inconsistent capitalization (sometimes first letter is capital)
- tweet_id should be an integer
- p1, p2, and p3 contain underscores instead of spaces in the labels


### Tidiness

- The 3 data frames twitter archive, images, and tweet count should be one combined table
- one variable in four columns (doggo, floofer, pupper, and puppo)
- no need for retweet columns (not investigating retweets)

## Cleaning: ( Define, Code, Test)

Here will fix the quality and tidiness issues that we identified in the assess step.

In [96]:
#copy dataframes
twitter_counts_clean = twitter_counts_df.copy()
twitter_archive_clean = twitter_archive.copy()
images_clean= images.copy()

### Tidiness Issues
**1- Mergeing Dataframes**
##### Code

In [97]:
# Merge the twitter_archive, twitter_counts_df, and images dataframes on 'tweet_id'
dfs = [twitter_archive_clean, twitter_counts_clean, images_clean]
twitter_clean = reduce(lambda left,right: pd.merge(left,right,on='tweet_id'), dfs)

##### Test

In [98]:
twitter_clean.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [99]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2073 non-null   int64  
 1   in_reply_to_status_id       23 non-null     float64
 2   in_reply_to_user_id         23 non-null     float64
 3   timestamp                   2073 non-null   object 
 4   source                      2073 non-null   object 
 5   text                        2073 non-null   object 
 6   retweeted_status_id         79 non-null     float64
 7   retweeted_status_user_id    79 non-null     float64
 8   retweeted_status_timestamp  79 non-null     object 
 9   expanded_urls               2073 non-null   object 
 10  rating_numerator            2073 non-null   int64  
 11  rating_denominator          2073 non-null   int64  
 12  name                        2073 non-null   object 
 13  doggo                       2073 

**2-Dog stages in One column**

 *Difine : Extracting dog stages from the 'text' column into the new 'dog_stage' column then droping the 'doggo', 'floofer', 'pupper', and 'puppo' columns. This is a tindiness and  quality issue of "name has values that are the string "None" instead of NaN".*
 
 ##### Code

In [126]:
twitter_clean.groupby(["doggo", "floofer", "pupper", "puppo"]).size().reset_index().rename(columns={0: "count"})

,doggo,floofer,pupper,puppo,count
0,None,None,None,None,1753
1,None,None,None,puppo,23
2,None,None,pupper,None,210
3,None,floofer,None,None,7
4,doggo,None,None,None,67
5,doggo,None,None,puppo,1
6,doggo,None,pupper,None,11
7,doggo,floofer,None,None,1


In [133]:
twitter_clean.doggo.replace('None', '', inplace=True)
twitter_clean.floofer.replace('None', '', inplace=True)
twitter_clean.pupper.replace('None', '', inplace=True)
twitter_clean.puppo.replace('None', '', inplace=True)
twitter_clean.doggo.replace(np.NaN, '', inplace=True)
twitter_clean.floofer.replace(np.NaN, '', inplace=True)
twitter_clean.pupper.replace(np.NaN, '', inplace=True)
twitter_clean.puppo.replace(np.NaN, '', inplace=True)

In [134]:

#twitter_clean.doggo 
twitter_clean['stage'] = twitter_clean.doggo + twitter_clean.floofer +twitter_clean.pupper + twitter_clean.puppo
twitter_clean.loc[twitter_clean.stage == 'doggopupper', 'stage'] = 'doggo, pupper'
twitter_clean.loc[twitter_clean.stage == 'doggopuppo', 'stage'] = 'doggo, puppo'
twitter_clean.loc[twitter_clean.stage == 'doggofloofer', 'stage'] = 'doggo, floofer'


In [135]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2073 non-null   int64  
 1   in_reply_to_status_id       23 non-null     float64
 2   in_reply_to_user_id         23 non-null     float64
 3   timestamp                   2073 non-null   object 
 4   source                      2073 non-null   object 
 5   text                        2073 non-null   object 
 6   retweeted_status_id         79 non-null     float64
 7   retweeted_status_user_id    79 non-null     float64
 8   retweeted_status_timestamp  79 non-null     object 
 9   expanded_urls               2073 non-null   object 
 10  rating_numerator            2073 non-null   int64  
 11  rating_denominator          2073 non-null   int64  
 12  name                        2073 non-null   object 
 13  doggo                       2073 

In [136]:
print(twitter_clean.stage.value_counts())

                  1753
pupper             210
doggo               67
puppo               23
doggo, pupper       11
floofer              7
doggo, floofer       1
doggo, puppo         1
Name: stage, dtype: int64


In [137]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2073 non-null   int64  
 1   in_reply_to_status_id       23 non-null     float64
 2   in_reply_to_user_id         23 non-null     float64
 3   timestamp                   2073 non-null   object 
 4   source                      2073 non-null   object 
 5   text                        2073 non-null   object 
 6   retweeted_status_id         79 non-null     float64
 7   retweeted_status_user_id    79 non-null     float64
 8   retweeted_status_timestamp  79 non-null     object 
 9   expanded_urls               2073 non-null   object 
 10  rating_numerator            2073 non-null   int64  
 11  rating_denominator          2073 non-null   int64  
 12  name                        2073 non-null   object 
 13  doggo                       2073 

**3- Unnecessary columns**

*Difine: there are unnecessary columns that we don't need in the analysis. They are 'in_reply_to_status_id', 
                  'in_reply_to_user_id', 'source'.*
                  
##### Code

In [58]:
twitter_clean.drop(['in_reply_to_status_id', 
                  'in_reply_to_user_id',
                  'source'], axis = 1, inplace=True)
twitter_clean.info()

KeyError: "['in_reply_to_status_id' 'in_reply_to_user_id' 'source'] not found in axis"

### Quality issues 
### **1-Remove retweets columns** (issue)

##### Code

In [59]:
# Keep the 'retweeted_status_id' value when it is NaN
twitter_clean = twitter_clean[np.isnan(twitter_clean.retweeted_status_id)]

# Check no non-null entries for 'retweeted_status_id', 'retweeted_status_user_id' and 'retweeted_status_timestamp'
print(twitter_clean.info())

# Remove 'retweeted_status_id', 'retweeted_status_user_id' and 
# 'retweeted_status_timestamp' 
twitter_clean = twitter_clean.drop(['retweeted_status_id', 
                        'retweeted_status_user_id', 
                        'retweeted_status_timestamp'], 
                       axis=1)

AttributeError: 'DataFrame' object has no attribute 'retweeted_status_id'

##### Test

In [60]:
#check the nucessary column had been removed
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174132 entries, 0 to 174131
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   tweet_id            174132 non-null  int64 
 1   timestamp           174132 non-null  object
 2   text                174132 non-null  object
 3   rating_numerator    174132 non-null  int64 
 4   rating_denominator  174132 non-null  int64 
 5   name                174132 non-null  object
 6   dogs_stage          133804 non-null  object
 7   dog_stage           28308 non-null   object
dtypes: int64(3), object(5)
memory usage: 10.6+ MB


### **2- tweet_id data type** (issue)

*Difine* : change the type from integer to string/object

##### Code

In [61]:
# change type 'tweet_id' from integer to a string
twitter_clean['tweet_id'] = twitter_clean['tweet_id'].astype(str)

##### Test

In [62]:
#checking the type 
twitter_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174132 entries, 0 to 174131
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   tweet_id            174132 non-null  object
 1   timestamp           174132 non-null  object
 2   text                174132 non-null  object
 3   rating_numerator    174132 non-null  int64 
 4   rating_denominator  174132 non-null  int64 
 5   name                174132 non-null  object
 6   dogs_stage          133804 non-null  object
 7   dog_stage           28308 non-null   object
dtypes: int64(2), object(6)
memory usage: 10.6+ MB


### **3- Create dog gender column**

*Define: extracting the gender of the dog from the 'text' column*

##### code

In [63]:
# Loop the texts and check if it has one of pronouns of male or female and append the result 

male = ['He', 'he', 'him', 'his', "he's", 'himself']
female = ['She', 'she', 'her', 'hers', 'herself', "she's"]

dog_gender = []

for text in twitter_clean['text']:
    # Male
    if any(map(lambda v:v in male, text.split())):
        dog_gender.append('male')
    # Female
    elif any(map(lambda v:v in female, text.split())):
        dog_gender.append('female')
    # If not specified
    else:
        dog_gender.append('NaN')

# Test
len(dog_gender)

# Save the result in a new column 'dog_name'
twitter_clean['dog_gender'] = dog_gender

In [64]:
print("dog_gender count \n", twitter_clean.dog_gender.value_counts())

dog_gender count 
 NaN       99036
male      54936
female    20160
Name: dog_gender, dtype: int64


### **4- Timestamp type**

*Define: change the type of the 'timestamp' column from object to a datetime object

##### Code

In [65]:
# Remove time-zone info from 'timestamp' 
twitter_clean['timestamp'] = twitter_clean['timestamp'].str.slice(start=0, stop=-6)
# Change the 'timestamp' column to a datetime object
twitter_clean['timestamp'] = pd.to_datetime(twitter_clean['timestamp'], format = "%Y-%m-%d %H:%M:%S")


##### Test

In [66]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174132 entries, 0 to 174131
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   tweet_id            174132 non-null  object        
 1   timestamp           174132 non-null  datetime64[ns]
 2   text                174132 non-null  object        
 3   rating_numerator    174132 non-null  int64         
 4   rating_denominator  174132 non-null  int64         
 5   name                174132 non-null  object        
 6   dogs_stage          133804 non-null  object        
 7   dog_stage           28308 non-null   object        
 8   dog_gender          174132 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 12.0+ MB


### **5- Name Column** (issue)

*Define : Lots of inaccurate dog names such as 'O', 'a', 'an' and so on. So will change the names to None as we don't know what's the rest of the names.*

##### Code

In [67]:
#Replacing the dog names with 'None':
twitter_clean.name = twitter_clean.name.str.replace('^[a-z]+', 'None')


##### Test

In [68]:
#Checking the names
twitter_clean.name.value_counts()

None       56868
Charlie      924
Tucker       840
Lucy         840
Penny        840
           ...  
Pluto         84
Benny         84
Brockly       84
Tug           84
BeBe          84
Name: name, Length: 914, dtype: int64

### **6- The rating exported wrongly**

*Define : some rating are inaccurate like in decimals 13.5 and exported in the wronge shape like 5. So we will replace the numerator values with the correct values.*

##### Code

In [69]:
# indices the text,, and ratings for tweets that contain a decimal in the numerator of rating
ratings_decimals_text = []
ratings_decimals_index = []
ratings_decimals = []

for i, text in twitter_clean['text'].iteritems():
    if bool(re.search('\d+\.\d+\/\d+', text)):
        ratings_decimals_text.append(text)
        ratings_decimals_index.append(i)
        ratings_decimals.append(re.search('\d+\.\d+', text).group())

# Print the text to confirm presence of ratings with decimals        
ratings_decimals_index

[40,
 558,
 614,
 1451,
 2113,
 2631,
 2687,
 3524,
 4186,
 4704,
 4760,
 5597,
 6259,
 6777,
 6833,
 7670,
 8332,
 8850,
 8906,
 9743,
 10405,
 10923,
 10979,
 11816,
 12478,
 12996,
 13052,
 13889,
 14551,
 15069,
 15125,
 15962,
 16624,
 17142,
 17198,
 18035,
 18697,
 19215,
 19271,
 20108,
 20770,
 21288,
 21344,
 22181,
 22843,
 23361,
 23417,
 24254,
 24916,
 25434,
 25490,
 26327,
 26989,
 27507,
 27563,
 28400,
 29062,
 29580,
 29636,
 30473,
 31135,
 31653,
 31709,
 32546,
 33208,
 33726,
 33782,
 34619,
 35281,
 35799,
 35855,
 36692,
 37354,
 37872,
 37928,
 38765,
 39427,
 39945,
 40001,
 40838,
 41500,
 42018,
 42074,
 42911,
 43573,
 44091,
 44147,
 44984,
 45646,
 46164,
 46220,
 47057,
 47719,
 48237,
 48293,
 49130,
 49792,
 50310,
 50366,
 51203,
 51865,
 52383,
 52439,
 53276,
 53938,
 54456,
 54512,
 55349,
 56011,
 56529,
 56585,
 57422,
 58084,
 58602,
 58658,
 59495,
 60157,
 60675,
 60731,
 61568,
 62230,
 62748,
 62804,
 63641,
 64303,
 64821,
 64877,
 65714,


In [70]:
# Change'rating_numerator' 
twitter_clean.loc[ratings_decimals_index[0],'rating_numerator'] = float(ratings_decimals[0])
twitter_clean.loc[ratings_decimals_index[1],'rating_numerator'] = float(ratings_decimals[1])
twitter_clean.loc[ratings_decimals_index[2],'rating_numerator'] = float(ratings_decimals[2])
twitter_clean.loc[ratings_decimals_index[3],'rating_numerator'] = float(ratings_decimals[3])

##### Test

In [71]:
twitter_clean.loc[1451]

tweet_id                                             680494726643068929
timestamp                                           2015-12-25 21:06:00
text                  Here we have uncovered an entire battalion of ...
rating_numerator                                                  11.26
rating_denominator                                                   10
name                                                               None
dogs_stage                                                          NaN
dog_stage                                                        pupper
dog_gender                                                          NaN
Name: 1451, dtype: object

In [72]:
twitter_clean.text.loc[1224]

"We normally don't rate unicorns but this one has 3 ears so it must be super rare. 12/10 majestic af https://t.co/f9qlKiv39T"

In [73]:
twitter_clean['rating_num'] = [int(round(num/(denom/10)))  if denom != 10 and num/denom <= 2 
                           else num for num, denom in zip(twitter_clean['rating_numerator'], twitter_clean['rating_denominator'])]
twitter_clean['rating_denom'] = [10 if denom != 10 and num/denom <= 2
                             else denom for num, denom in zip(twitter_clean['rating_numerator'], twitter_clean['rating_denominator'])]
# drop other columns
#df_master = df_master.drop(['rating_numerator', 'rating_denominator'], axis=1)

### **7- Numerator & Denominator data type**

*Define : Numerator rate has some floating content so will change the column type from object to float*

##### Code

In [74]:
twitter_clean['rating_numerator'] = twitter_clean['rating_numerator'].astype(float)
twitter_clean['rating_denominator'] = twitter_clean['rating_denominator'].astype(float)

##### Test

In [75]:
twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174132 entries, 0 to 174131
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   tweet_id            174132 non-null  object        
 1   timestamp           174132 non-null  datetime64[ns]
 2   text                174132 non-null  object        
 3   rating_numerator    174132 non-null  float64       
 4   rating_denominator  174132 non-null  float64       
 5   name                174132 non-null  object        
 6   dogs_stage          133804 non-null  object        
 7   dog_stage           28308 non-null   object        
 8   dog_gender          174132 non-null  object        
 9   rating_num          174132 non-null  float64       
 10  rating_denom        174132 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 14.6+ MB


 ### **8- The rating in 'text' is inaccurate**
 
 *Define: rating  aren't right where there are many signs that affect the rating and confuse the analyst*
 
 ##### Code
 

In [76]:
# Fix the text where there are more than one '/'
text_ratings_to_fix = twitter_clean[twitter_clean.text.str.contains( r"(\d+\.?\d*\/\d+\.?\d*\D+\d+\.?\d*\/\d+\.?\d*)")].text

text_ratings_to_fix

/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


454       RT @dog_rates: "Yep... just as I suspected. Yo...
616       "Yep... just as I suspected. You're not flossi...
825       This is Bookstore and Seaweed. Bookstore is ti...
876       After so many requests, this is Bretagne. She ...
967       Happy 4/20 from the squad! 13/10 for all https...
                                ...                        
173993    This is Spark. He's nervous. Other dog hasn't ...
174040    This is Kial. Kial is either wearing a cape, w...
174049    Two dogs in this one. Both are rare Jujitsu Py...
174082    These are Peruvian Feldspars. Their names are ...
174111    This is an Albanian 3 1/2 legged  Episcopalian...
Name: text, Length: 2352, dtype: object

In [77]:
for entry in text_ratings_to_fix:
    mask = twitter_clean.text == entry
    column_name1 = 'rating_numerator'
    column_name2 = 'rating_denominator'
    twitter_clean.loc[mask, column_name1] = re.findall(r"\d+\.?\d*\/\d+\.?\d*\D+(\d+\.?\d*)\/\d+\.?\d*", entry)
    twitter_clean.loc[mask, column_name2] = 10

In [78]:
twitter_clean[twitter_clean.text.isin(text_ratings_to_fix)]

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,dogs_stage,dog_stage,dog_gender,rating_num,rating_denom
454,802624713319034886,2016-11-26 21:26:58,"RT @dog_rates: ""Yep... just as I suspected. Yo...",11,10.0,None,NaN,NaN,NaN,12.0,10
616,777684233540206592,2016-09-19 01:42:24,"""Yep... just as I suspected. You're not flossi...",11,10.0,None,NaN,NaN,NaN,12.0,10
825,747600769478692864,2016-06-28 01:21:27,This is Bookstore and Seaweed. Bookstore is ti...,7,10.0,Bookstore,NaN,NaN,NaN,10.0,10
876,740373189193256964,2016-06-08 02:41:38,"After so many requests, this is Bretagne. She ...",14,10.0,None,NaN,NaN,female,8.0,10
967,722974582966214656,2016-04-21 02:25:47,Happy 4/20 from the squad! 13/10 for all https...,13,10.0,None,NaN,NaN,NaN,2.0,10
...,...,...,...,...,...,...,...,...,...,...,...
173993,668537837512433665,2015-11-22 21:13:35,This is Spark. He's nervous. Other dog hasn't ...,1,10.0,Spark,None,NaN,NaN,8.0,10
174040,667544320556335104,2015-11-20 03:25:43,"This is Kial. Kial is either wearing a cape, w...",4,10.0,Kial,None,NaN,NaN,10.0,10
174049,667491009379606528,2015-11-19 23:53:52,Two dogs in this one. Both are rare Jujitsu Py...,8,10.0,None,None,NaN,NaN,7.0,10
174082,666835007768551424,2015-11-18 04:27:09,These are Peruvian Feldspars. Their names are ...,10,10.0,None,None,NaN,NaN,10.0,10


In [79]:
twitter_clean.loc[2052]

tweet_id                                             666287406224695296
timestamp                                           2015-11-16 16:11:11
text                  This is an Albanian 3 1/2 legged  Episcopalian...
rating_numerator                                                      9
rating_denominator                                                   10
name                                                               None
dogs_stage                                                          NaN
dog_stage                                                           NaN
dog_gender                                                          NaN
rating_num                                                            5
rating_denom                                                         10
Name: 2052, dtype: object

In [80]:
from IPython.display import Image 
Image(url = ' https://pbs.twimg.com/media/CT8g3BpUEAAuFjg.jpg')

### **9- Missing Enteries in 'expanded_urls' columns**

*Define: Some enteries are missing from the column' expanded_urls' and will regenerate it using the tweet_id's*

##### Code


In [81]:
#count the missing enteries 
twitter_clean['expanded_urls'].isnull().count()

KeyError: 'expanded_urls'

In [ ]:
twitter_clean['expanded_urls'] = [str('https://twitter.com/dog_rates/status/' + str(id)) 
                              for id, url in zip(twitter_clean['tweet_id'], twitter_clean['expanded_urls'])]

###### Test

In [ ]:
assert twitter_clean['expanded_urls'].isnull().any() == False

#### 10- Creating Dog Breed column

##### Code

In [ ]:
twitter_clean['dog_breed'] = [i['p1'] if i['p1_dog'] == True and i['p1_conf'] > 0.2 
                     else i['p2'] if i['p2_dog'] == True and i['p2_conf'] > 0.2
                     else i['p3'] if i['p3_dog'] == True and i['p3_conf'] > 0.2
                     else np.nan for index, i in twitter_clean.iterrows()]

##### Test

In [ ]:
twitter_clean.info()

In [ ]:
twitter_clean.dog_breed.value_counts()

#### 11- Deleting Unnecessery columns

##### Code

In [ ]:
twitter_clean = twitter_clean.drop(['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], 1)

##### Test

In [ ]:
twitter_clean.info()

## Storing Data

In [ ]:
# Storing the new twitter_clean  to 'twitter_archive_master.csv' file
twitter_clean.to_csv('twitter_archive_master.csv', encoding='utf-8', index=False)

## Exploration and Visualizetion Data Analysis

In [ ]:
twitter_clean.describe()

In [ ]:
twitter_clean.timestamp = pd.to_datetime(twitter_clean['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

monthly_tweets = twitter_clean.groupby(pd.Grouper(key = 'timestamp', freq = "M")).count().reset_index()
monthly_tweets = monthly_tweets[['timestamp', 'tweet_id']]
monthly_tweets.head()
monthly_tweets.sum()

In [ ]:
twitter_clean.info()

##### Visalizing total tweet over time 

In [ ]:
# Tweets vs. time plot 
plt.figure(figsize=(10, 8))

plt.xlabel('Year and Month')
plt.ylabel('Tweets Count')
plt.grid()
plt.plot(monthly_tweets.timestamp, monthly_tweets.tweet_id)
plt.title('WeRateDogs Tweets over Time');

In [ ]:
twitter_clean.info()

#### Counts of retweets vs counts of favorite 

In [ ]:
sns.lmplot(x="retweet_count", 
           y="favorite_count", 
           data=twitter_clean,
           height = 5,
           aspect=1.3,
           scatter_kws={'alpha':1/5});
plt.grid()
plt.title('Favorite Count vs. Retweet Count');
plt.xlabel('Retweet Count');
plt.ylabel('Favorite Count');

### Most famous dog breed  

In [ ]:
breed_groups = twitter_clean.groupby('dog_breed')['rating_num'].mean()

# filters out dog breed that appear less than 10 times
breed_groups = breed_groups[twitter_clean['dog_breed'].value_counts() > 10].sort_values(ascending=False)
breed_groups

In [ ]:
# define axis values
x_axis = list(breed_groups)
y_axis = [i.title() for i in breed_groups.index]

# plot the bar chart
plt.figure(figsize=[10,14])
plt.xlim(8,13)
plt.xlabel('Average Rating')
plt.title('Average Rating for each Dog Breed on WeRateDogs™')
colors = ['#8dafbe' if (x < max(x_axis)) else '#f8e863' for x in x_axis]
ax = sns.barplot(x=x_axis, y=y_axis, label=y_axis, color="b", palette=colors)
plt.show();

**The chart shows that hightest rate dog breed is the Eskimo Dog**

#### Most Favorite Dog Name

In [ ]:
dog_name = twitter_clean[twitter_clean['name'] != 'None']
dog_name.name.value_counts()[0:10].plot('barh', 
                                            figsize=(15,8), 
                                            title='Most Common Dog Names').set_xlabel("Number of Dogs");

**The chart shows that Charlie is the most favorite name **

In [ ]:
dog_name.name.value_counts()